In [1]:
import pandas as pd
import datetime as dt
import os

# CORAL Introduction

CORAL uses a pool of allocations and a pipeline of projects to estimate the delays incurred by limited port and vessel resources. This tutorial shows the basic workflow for running CORAL. For running and processing multiple runs, we reccommend reading documentation on run_coral.py and coral_postproc.py.

In [2]:
# import ORBIT path
import sys
sys.path.insert(0, '/Users/sbredenk/Repos/ORBIT')

# import CORAL modules
from CORAL import SharedLibrary, GlobalManager, Pipeline

## Core Inputs

The required inputs include base configuration files for both fixed and floating. These are included in the analysis/configs folder (but feel free to make your own). Other required inputs include the filepath to the library folder which holds cable, port, turbine, vessel, and weather configuration files, and the pipeline file which holds all project specific information. Finally, the initial allocations of port and vessel resources must be given in a dictionary as shown below. The first value in each tuple is the vessel name as defined in the vessel library and the second is the number of vessels of that type. 

In [3]:
# paths
base_fixed_config = os.path.join(os.getcwd(), "analysis", "configs", "base.yaml")
base_floating_config = os.path.join(os.getcwd(), "analysis", "configs", "base_float.yaml")
library_fp = os.path.join(os.getcwd(), "analysis", "library")
pipeline_fp = os.path.join(os.getcwd(), "analysis", "pipelines", "coral_ex_pipeline.csv")

# allocations
allocations = {
    "wtiv": [('example_heavy_lift_vessel', 2),('example_wtiv', 2)],
    "feeder": [('example_feeder', 6), ('example_heavy_feeder_1kit', 6)],
    "port": [('new_london', 1), ('new_bedford', 1), ('sbmt', 1), ('njwp', 1)],
    "ahts_vessel": ('example_ahts_vessel', 2),
    "towing_vessel": ('example_towing_vessel', 2),
}

## Optional Inputs

The two main optional inputs are weather profiles and future resources. The weather files are held in analysis/library/weather (an example weather profile is included). The future port or vessel resources are defined as a list of dates the new resources will be available. If multiple dates are given, one additional resource will be added at each date in the list.

In [4]:
# weather
weather_fp = os.path.join(os.getcwd(), "analysis", "library", "weather", "vineyard_wind_repr_with_whales.csv")
weather = pd.read_csv(weather_fp, parse_dates=["datetime"]).set_index("datetime")

# future resources: define dates for future resource to join the pool
wtiv_future_dates = [dt.date(2025,1,1), dt.date(2030,1,1)]
sbmt_future_dates = [dt.date(2032,1,1)]

## Running the Model

To run CORAL, first create your pipeline using the Pipeline module as shown below. Then run GlobalManager using your pipeline and allocations. Add all future resources using the add_future_resources function, giving the resource class, resource name, and list of dates defined above.

In [5]:
pipeline = Pipeline(pipeline_fp, base_fixed_config, base_floating_config)
manager = GlobalManager(pipeline.configs, allocations, library_path=library_fp)
manager.add_future_resources('wtiv', 'example_wtiv', wtiv_future_dates)
manager.add_future_resources('port', 'sbmt', sbmt_future_dates)
manager.run()

ORBIT library intialized at 'c:\Users\sbredenk\Repos\ORBIT_natl_gaps\library'
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:
TurbineInstallation:


## Output DataFrame

In [6]:
df = pd.DataFrame(manager.logs).iloc[::-1]
df = df.reset_index(drop=True).reset_index()
display(df)

,index,name,Initialized,Started,Finished,Date Initialized,Date Started,Date Finished
0,0,Project 19,61368,127592.020534,135562.469724,2030-01-01,2037-07-22 09:00:00,2038-06-19 11:00:00
1,1,Project 15,52608,117725.954580,127592.020534,2029-01-01,2036-06-06 06:00:00,2037-07-22 09:00:00
2,2,Project 13,52608,107859.888625,117725.954580,2029-01-01,2035-04-22 04:00:00,2036-06-06 06:00:00
3,3,Project 11,52608,93193.911271,107859.888625,2029-01-01,2033-08-19 02:00:00,2035-04-22 04:00:00
4,4,Project 9,43824,86299.352834,93193.911271,2028-01-01,2032-11-04 20:00:00,2033-08-19 02:00:00
5,5,Project 17,61368,70940.124639,86299.352834,2030-01-01,2031-02-03 21:00:00,2032-11-04 20:00:00
6,6,Project 20,61368,76418.575975,84465.935134,2030-01-01,2031-09-20 03:00:00,2032-08-20 10:00:00
7,7,Project 18,61368,61368.000000,76418.575975,2030-01-01,2030-01-01 00:00:00,2031-09-20 03:00:00
8,8,Project 16,61368,61368.000000,71558.159429,2030-01-01,2030-01-01 00:00:00,2031-03-01 15:00:00
9,9,Project 7,35064,60880.697335,70940.124639,2027-01-01,2029-12-11 17:00:00,2031-02-03 21:00:00
